In [1]:
#Mount google drive to import dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 40.1 MB/s eta 0:00:00


In [5]:
#Verify that that YOLO model was imported successfully
from ultralytics import YOLO

print("Ultralytics imported OK")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics imported OK


In [8]:
#Set up a pointer to the path of the dataset (data.yaml)
#(data.yaml tells YOLO where train/val images and labels are and what the class names are (e.g., resistor, capacitor, etc.)
DATA_YAML = r"/content/drive/MyDrive/Circuit-Diagram-CNN/Circuit Component Detection/data.yaml"
print(DATA_YAML)

/content/drive/MyDrive/Circuit-Diagram-CNN/Circuit Component Detection/data.yaml


In [9]:
#We can open the file to see its contents
with open(DATA_YAML, "r") as f:
    print(f.read())

#The yaml file is essentially the dataset config file. It tells the model where the images are,
#which split is train/validation/test, and what the classes are and how they are indexed.

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 6
names: ['Capacitor', 'GND', 'I_dc', 'Inductor', 'Resistor', 'V_dc']

roboflow:
  workspace: iut-uf4xg
  project: circuit-component-detection-9wiz7
  version: 4
  license: CC BY 4.0
  url: https://universe.roboflow.com/iut-uf4xg/circuit-component-detection-9wiz7/dataset/4


In [10]:
#Load a pretrained YOLOv8 model (nano model)
model = YOLO("yolov8n.pt")

#Train on your dataset
results = model.train(
    data=DATA_YAML,   # path to data.yaml
    epochs=100,       # how many epochs to train the model through
    imgsz=640,        # image size
    batch=16,         # how many batches per epoch during training
)


Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Circuit-Diagram-CNN/Circuit Component Detection/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patienc

In [14]:
#Validate on best weights and print metrics

#best.pt holds a checkpoint of the model when it had the best validation mAP during training
#
best_weights = "runs/detect/train/weights/best.pt"

model = YOLO(best_weights)

metrics = model.val(
    data=DATA_YAML,
    split="val"
)

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 18.9±2.0 MB/s, size: 17.7 KB)
val: Scanning /content/drive/MyDrive/Circuit-Diagram-CNN/Circuit Component Detection/valid/labels.cache... 7 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 7/7 19.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 3.9it/s 0.3s
                   all          7         51      0.953      0.953      0.986      0.684
             Capacitor          7          8       0.98          1      0.995      0.632
                   GND          7          7       0.98          1      0.995      0.719
                  I_dc          7          7          1      0.817      0.995      0.733
              Inductor          7         10      0.974        0.9       0.94      0.605
              Res

**What these metrics mean:**

The key lines are...

```
# Class     Images  Instances      Box(P)      R      mAP50  mAP50-95
# all            7         51      0.953    0.953     0.986     0.684
```

* Precision ≈ 0.95 (When the model predicts a component, it’s correct about 95% of the time.)

* Recall ≈ 0.95 (It finds about 95% of all true components in those 7 validation images.)

* mAP0.5 ≈ 0.99 (Very high; almost perfect detection.)

* mAP@0.5-0.95 ≈ 0.68 (This is a stricter quality measure. ~0.68 on a tiny dataset is still pretty solid.)

In [16]:
#Test the model on test circuit diagram images to check symbol detection performance
model = YOLO("runs/detect/train/weights/best.pt")

test_metrics = model.val(
    data=DATA_YAML,
    split="test"
    )

Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 17.1±1.3 MB/s, size: 16.2 KB)
val: Scanning /content/drive/MyDrive/Circuit-Diagram-CNN/Circuit Component Detection/test/labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 5/5 12.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 5.7it/s 0.2s
                   all          5         32      0.956      0.952      0.954      0.714
             Capacitor          5          7      0.998          1      0.995      0.707
                   GND          4          4      0.961          1      0.995      0.709
              Inductor          5          5      0.929        0.8      0.843      0.667
              Resistor          5          9      0.891          1       0.94      0.674
                  

**Interpreting the data:**

* Overall precision ≈ 0.96 (96% of detections are correct.)

* Overall recall ≈ 0.95 (The model finds ~95% of the components.)

* mAP@0.5 ≈ 0.95 (Very good object detection performance at IoU ≥ 0.5.)

* mAP@0.5–0.95 ≈ 0.71 (This is the stricter metric; 0.7+ on a tiny dataset is solid.)

**Per-class evaluations:**

* Capacitor / GND (Ground) / V_dc (Voltage source) look very strong.

* Resistor is slightly weaker in precision (0.89) but recall is 1.0.

* Inductor is the “weakest” class: recall 0.8, mAP50 0.84, mAP50-95 0.67, but is still decent. (If something were to fail, it will probably be inductors)

In [ ]:
#Test on our own hand drawn circuit diagrams